# Criação de Tabela com População Total de cada Estado

O problema começa quando o GeoJSON foi indexado com uma id que vale pelo Municipio todo, por exemplo id = 1200013 é do Acre (12) e Municipio(13) que é Acrelandia.

Porém para adicionar o dado da população total é necessario utilizar duas tabelas para fazer esse calculo. As tabelas são, DescriçãoXX.xls e PESSOAP1_XX.xls. A primeira tabela respectivamente contém o dado de chave (id de cada municipio), o id do Municipio, Nome do Distrito entre outros. (o que importa é a chave e o id do municipio). A segunda tabela contém o codigo do setor (igual a chave da primeira tabela) e o valor da população total dos setores do Municipio.

É preciso juntar as duas tabelas, em uma só tabela que contenha o id novo (Esse id é o codigo do estado + Municipio), População total do municipio.

In [1]:
# -*- coding: utf-8 -*-
# Importar bibliotecas necessarias:
%matplotlib inline
import numpy as np
import glob
import pandas as pd
from pandas import DataFrame

In [2]:
# Lista de todas os Estados contidos na pasta de Dado Censitarios

Lista_paths = glob.glob('/Users/samfs/Documents/Projeto1_netmapbr/Dado_Censitarios_2007/*')
Lista_paths

['/Users/samfs/Documents/Projeto1_netmapbr/Dado_Censitarios_2007\\11',
 '/Users/samfs/Documents/Projeto1_netmapbr/Dado_Censitarios_2007\\12',
 '/Users/samfs/Documents/Projeto1_netmapbr/Dado_Censitarios_2007\\13',
 '/Users/samfs/Documents/Projeto1_netmapbr/Dado_Censitarios_2007\\14',
 '/Users/samfs/Documents/Projeto1_netmapbr/Dado_Censitarios_2007\\15',
 '/Users/samfs/Documents/Projeto1_netmapbr/Dado_Censitarios_2007\\16',
 '/Users/samfs/Documents/Projeto1_netmapbr/Dado_Censitarios_2007\\17',
 '/Users/samfs/Documents/Projeto1_netmapbr/Dado_Censitarios_2007\\21',
 '/Users/samfs/Documents/Projeto1_netmapbr/Dado_Censitarios_2007\\22',
 '/Users/samfs/Documents/Projeto1_netmapbr/Dado_Censitarios_2007\\23',
 '/Users/samfs/Documents/Projeto1_netmapbr/Dado_Censitarios_2007\\24',
 '/Users/samfs/Documents/Projeto1_netmapbr/Dado_Censitarios_2007\\25',
 '/Users/samfs/Documents/Projeto1_netmapbr/Dado_Censitarios_2007\\26',
 '/Users/samfs/Documents/Projeto1_netmapbr/Dado_Censitarios_2007\\27',
 '/Use

 Extrair para cada pasta apenas as duas tabelas de interesse:  Descrição de setores e PESSOAP1.

In [3]:
Lista_descricao = []
Lista_pessoas = []
for path in Lista_paths:
    Lista_descricao.append( glob.glob(path + '/Descrição de setores*.xls') )
    Lista_pessoas.append( glob.glob(path + '/PESSOAP1*.xls') )
print(  ("\n Arquivos de PESSOAP1 = {} \n \n Arquivos de Descricao de setores = {}").format(Lista_pessoas,Lista_descricao)  )


 Arquivos de PESSOAP1 = [['/Users/samfs/Documents/Projeto1_netmapbr/Dado_Censitarios_2007\\11\\PESSOAP1_11.xls'], ['/Users/samfs/Documents/Projeto1_netmapbr/Dado_Censitarios_2007\\12\\PESSOAP1_12.xls'], ['/Users/samfs/Documents/Projeto1_netmapbr/Dado_Censitarios_2007\\13\\PESSOAP1_13.xls'], ['/Users/samfs/Documents/Projeto1_netmapbr/Dado_Censitarios_2007\\14\\PESSOAP1_14.xls'], ['/Users/samfs/Documents/Projeto1_netmapbr/Dado_Censitarios_2007\\15\\PESSOAP1_15.xls'], ['/Users/samfs/Documents/Projeto1_netmapbr/Dado_Censitarios_2007\\16\\PESSOAP1_16.xls'], ['/Users/samfs/Documents/Projeto1_netmapbr/Dado_Censitarios_2007\\17\\PESSOAP1_17.xls'], ['/Users/samfs/Documents/Projeto1_netmapbr/Dado_Censitarios_2007\\21\\PESSOAP1_21.xls'], ['/Users/samfs/Documents/Projeto1_netmapbr/Dado_Censitarios_2007\\22\\PESSOAP1_22.xls'], ['/Users/samfs/Documents/Projeto1_netmapbr/Dado_Censitarios_2007\\23\\PESSOAP1_23.xls'], ['/Users/samfs/Documents/Projeto1_netmapbr/Dado_Censitarios_2007\\24\\PESSOAP1_24.xl

## Para cada estado criar a nova tabela e armazenar em um DataFrame:

In [4]:
from sqlalchemy import create_engine
engine = create_engine('sqlite://', echo=False)

# Para cada arquivo do estado criar nova tabela
COLUNAS = [
    'id',
    'idGeo',
    'UF',
    'Munic',
    'Distr',
    'Setor',
    'Situacao',
    'NUF',
    'NMunic',
    'NDistr',
    'Populacao'
]

df = pd.DataFrame(columns=COLUNAS)
df.head()

cnt = 0
for (descricao,pessoa) in zip(Lista_descricao,Lista_pessoas):
    
    tabela_pessoa = pd.read_excel(str(pessoa[0]))
    tabela_descricao = pd.read_excel(str(descricao[0]))
    
    # Selecionar os campos
    tabela_descricao = tabela_descricao.loc[:,['Chave','UF','Munic','Distr','Setor','NUF','NMunic','NDistr','NSituacao']]
    tabela_pessoa = tabela_pessoa = tabela_pessoa.loc[:,['Cod_setor','V0301']]
    
    #Transformar para sql
    tabela_pessoa.to_sql('pessoa_sql4' + str(cnt), con=engine)
    tabela_descricao.to_sql('descricao_sql4' + str(cnt),con=engine)
    
    query_sql = "SELECT da.Chave, da.UF, da.Munic, da.Distr, da.Setor, da.NSituacao ,da.NUF, da.NMunic, da.NDistr , pa.V0301 FROM " + "descricao_sql4" + str(cnt) +  " AS da LEFT JOIN " + "pessoa_sql4" + str(cnt) + " AS pa ON da.Chave = pa.Cod_setor"
     
    df_estado = DataFrame(engine.execute(query_sql).fetchall())
    
    # Tabela intermediaria
    df_estado.columns = ['id','UF', 'Munic', 'Distr', 'Setor', 'Situacao', 'NUF' , 'NMunic', 'NDistr', 'Populacao']
    
    # Criar coluna para identifica no mapa GeoJSON
    df_estado['idGeo'] = '0'
    for b in df_estado.itertuples():
        if( b.Munic<100):
            df_estado.idGeo[b.Index] = str(b.UF) + '000' + str(b.Munic)
        elif(b.Munic<1000):
            df_estado.idGeo[b.Index] = str(b.UF) + '00' + str(b.Munic)
        elif(b.Munic<10000):
            df_estado.idGeo[b.Index] = str(b.UF) + '0' + str(b.Munic)
        else:
            df_estado.idGeo[b.Index] = str(b.UF) + str(b.Munic)
            
    
    # Organizar colunas:
    df_estado = df_estado[['id','idGeo','UF','Munic','Distr','Setor','Situacao','NUF','NMunic','NDistr','Populacao']]

    # Concatenar com Dataframe final:
    df = pd.concat([df, df_estado])
    
    cnt+=1
    
    
    

C:\Users\samfs\Anaconda3\lib\site-packages\ipykernel_launcher.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\samfs\Anaconda3\lib\site-packages\ipykernel_launcher.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\samfs\Anaconda3\lib\site-packages\ipykernel_launcher.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\samfs\Anaconda3\lib\site-packages\ipykernel_launcher.py:53: SettingWithCopyWarning: 
A value i

In [7]:
df.head(10)
df.loc[(df.Setor == 20)]

,id,idGeo,UF,Munic,Distr,Setor,Situacao,NUF,NMunic,NDistr,Populacao
19,110001505000020,1100015,11,15,5,20,80 - Rural,Rondônia,ALTA FLORESTA D'OESTE,ALTA FLORESTA D'OESTE,434.0
77,110002305000020,1100023,11,23,5,20,10 - Urbana,Rondônia,ARIQUEMES,ARIQUEMES,1034.0
202,110004905000020,1100049,11,49,5,20,10 - Urbana,Rondônia,CACOAL,CACOAL,730.0
281,110004910000020,1100049,11,49,10,20,80 - Rural,Rondônia,CACOAL,RIOZINHO,357.0
356,110080905000020,1100809,11,809,5,20,80 - Rural,Rondônia,CANDEIAS DO JAMARI,CANDEIAS DO JAMARI,582.0
...,...,...,...,...,...,...,...,...,...,...,...
7198,522060305000020,5220603,52,20603,5,20,80 - Rural,Goiás,SILVÂNIA,SILVÂNIA,265.0
7273,522140305000020,5221403,52,21403,5,20,10 - Urbana,Goiás,TRINDADE,TRINDADE,2185.0
7390,522160105000020,5221601,52,21601,5,20,10 - Urbana,Goiás,URUAÇU,URUAÇU,746.0
7440,522170005000020,5221700,52,21700,5,20,80 - Rural,Goiás,URUANA,URUANA,96.0


# Eliminar valores NaN por 0

In [8]:
# for column
df['Populacao'] = df['Populacao'].replace(np.nan, 0)

df.loc[(df.Setor == 10) & (df.UF == 29) & (df.Munic > 5700)]

,id,idGeo,UF,Munic,Distr,Setor,Situacao,NUF,NMunic,NDistr,Populacao
1314,290640205000010,2906402,29,6402,5,10,50 - Rural,Bahia,CANDEAL,CANDEAL,33.0
1323,290830915000010,2908309,29,8309,15,10,80 - Rural,Bahia,CONCEIÇÃO DO ALMEIDA,RIO DA DONA,325.0
1333,290840805000010,2908408,29,8408,5,10,10 - Urbana,Bahia,CONCEIÇÃO DO COITÉ,CONCEIÇÃO DO COITÉ,832.0
2226,290570105000010,2905701,29,5701,5,10,10 - Urbana,Bahia,CAMAÇARI,CAMAÇARI,0.0
2387,290570110000010,2905701,29,5701,10,10,10 - Urbana,Bahia,CAMAÇARI,ABRANTES,0.0
...,...,...,...,...,...,...,...,...,...,...,...
18161,293340605000010,2933406,29,33406,5,10,80 - Rural,Bahia,WAGNER,WAGNER,301.0
18176,293345505000010,2933455,29,33455,5,10,50 - Rural,Bahia,WANDERLEY,WANDERLEY,467.0
18205,293350505000010,2933505,29,33505,5,10,80 - Rural,Bahia,WENCESLAU GUIMARÃES,WENCESLAU GUIMARÃES,1125.0
18247,293360405000010,2933604,29,33604,5,10,10 - Urbana,Bahia,XIQUE-XIQUE,XIQUE-XIQUE,558.0


# Formular uma tabela de estado por municipio com populacao total

In [9]:
tabela_total = df.copy()

tabela_total.to_sql('total_sql', con=engine)

In [10]:
query_sql = "SELECT idGeo, SUM(Populacao) FROM total_sql GROUP BY idGeo"
     
df_estado_total = DataFrame(engine.execute(query_sql).fetchall())

In [11]:
df_estado_total.columns = ['id_UF','Populacao']
df_estado_total

,id_UF,Populacao
0,1100015,23857.0
1,1100023,82388.0
2,1100031,6575.0
3,1100049,76155.0
4,1100056,16290.0
...,...,...
5558,5221908,3649.0
5559,5222005,12187.0
5560,5222054,5919.0
5561,5222203,4198.0


In [12]:
# Reindexei para cada

df_estado_total.set_index('id_UF', inplace=True)
df_estado_total.head(10)

,Populacao
id_UF,
1100015,23857.0
1100023,82388.0
1100031,6575.0
1100049,76155.0
1100056,16290.0
1100064,17644.0
1100072,9476.0
1100080,13664.0
1100098,27867.0


In [13]:
# Acesse population via index:
pop = df_estado_total.loc[ '1100098' ]
a = pop.values.tolist()
a[0]

27867.0

# Criar Novo GeoJSON por Estado

O novo GeoJSON contém a seguinte estrutura de JSON na parte de properties, não contem o wifi_ratio e wifi_count, pois esse arquivo eh um teste apenas:

"properties": { 

                "id": "1200013",
                
                "name": "Acrelandia",
                
                "description": "Acrelandia",
                
                "Populacao": "1082"
                
               }

In [17]:
# Update the GeoJSON Est. do Brasil

import json

    
# Read JSON:
geojson = glob.glob('/Users/samfs/Documents/Projeto1_netmapbr/GeoJSON/geojs-100-mu.json')

with open(geojson[0]) as json_file:
    data = json.load(json_file)


for feat in data['features']:
    idd = feat['properties']['id']
    if(idd == '5300108'):
        feat['properties']['Populacao'] = '2455903'
    else:
        valor = df_estado_total.loc[ idd , : ]
        a = valor.values.tolist()
        feat['properties']['Populacao'] = a[0]


with open('mapa_estado_brasil.json', 'w') as json_file:
    json.dump(data, json_file, ensure_ascii=False)


# Plotar um Mapa demonstrativo

In [18]:
county_geojson = json.load(open('GeoJSON/Estados_Corrigidos/Amazonas/geojs-13-mun.json'))

FileNotFoundError: [Errno 2] No such file or directory: 'GeoJSON/Estados_Corrigidos/Amazonas/geojs-13-mun.json'

In [19]:
m = folium.Map(location=[48, -102], zoom_start=3)

folium.Choropleth(
    geo_data=county_geojson,
    name='choropleth',
    data=df,
    columns=['idGeo', 'Populacao'],
    key_on='properties.id',
    fill_color='YlGn',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Population'
).add_to(m)

folium.LayerControl().add_to(m)

m

NameError: name 'folium' is not defined